# KFold Cross Validation Python Tutorial

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.datasets import load_digits
digits = load_digits()

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(digits.data,digits.target,test_size=0.3)

## Logistic Regression

In [3]:
lr = LogisticRegression(solver='liblinear',multi_class='ovr')
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

0.9722222222222222

## SVM

In [4]:
svm = SVC(gamma='auto')
svm.fit(X_train, y_train)
svm.score(X_test, y_test)

0.35

# Random Forest

In [5]:
rf = RandomForestClassifier(n_estimators=40)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.9851851851851852

# KFold cross validation

In [6]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)
kf

KFold(n_splits=3, random_state=None, shuffle=False)

In [7]:
for train_index, test_index in kf.split([1,2,3,4,5,6,7,8,9]):
    print(train_index, test_index)

[3 4 5 6 7 8] [0 1 2]
[0 1 2 6 7 8] [3 4 5]
[0 1 2 3 4 5] [6 7 8]


## Use KFold for our digits example

In [8]:
def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

In [16]:
get_score(LogisticRegression(solver='liblinear',multi_class='ovr'), X_train, X_test, y_train, y_test)

0.9722222222222222

In [9]:
get_score(SVC(), X_train, X_test, y_train, y_test)

0.9888888888888889

In [18]:
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=3)

In [19]:
scores_logistic = []
scores_svm = []
scores_rf = []

for train_index, test_index in folds.split(digits.data, digits.target):
    X_train, X_test, y_train, y_test = digits.data[train_index], digits.data[test_index], \
                                       digits.target[train_index], digits.target[test_index]
    scores_logistic.append(get_score(LogisticRegression(solver='liblinear',multi_class='ovr'), X_train, X_test, y_train, y_test))  
    scores_svm.append(get_score(SVC(gamma='auto'), X_train, X_test, y_train, y_test))
    scores_rf.append(get_score(RandomForestClassifier(n_estimators=40), X_train, X_test, y_train, y_test))

In [20]:
scores_logistic

[0.8948247078464107, 0.9532554257095158, 0.9098497495826378]

In [21]:
scores_svm

[0.3806343906510851, 0.41068447412353926, 0.5125208681135225]

In [22]:
scores_rf

[0.9298831385642737, 0.9415692821368948, 0.9198664440734557]

# cross_val_score function

In [23]:
from sklearn.model_selection import cross_val_score

## Logistic regression model performance using cross_val_score

In [24]:
cross_val_score(LogisticRegression(solver='liblinear',multi_class='ovr'), digits.data, digits.target, cv=3)

array([0.89482471, 0.95325543, 0.90984975])

## svm model performance using cross_val_score

In [25]:
cross_val_score(SVC(gamma='auto'), digits.data, digits.target,cv=3)

array([0.38063439, 0.41068447, 0.51252087])

## random forest performance using cross_val_score

In [26]:
cross_val_score(RandomForestClassifier(n_estimators=40),digits.data, digits.target,cv=3)

array([0.93489149, 0.93489149, 0.92988314])

cross_val_score uses stratifield kfold by default

# Parameter tunning using k fold cross validation

In [28]:
scores = []
estimators = [5, 20, 30, 40]

for i in range(len(estimators)):
    scores.append(cross_val_score(RandomForestClassifier(n_estimators=estimators[i]), digits.data, digits.target, cv=10))
scores

[array([0.80555556, 0.94444444, 0.89444444, 0.76666667, 0.88888889,
        0.91111111, 0.89444444, 0.93854749, 0.87150838, 0.84357542]),
 array([0.90555556, 0.96666667, 0.91111111, 0.9       , 0.95555556,
        0.97222222, 0.95      , 0.95530726, 0.90502793, 0.92178771]),
 array([0.88888889, 0.96666667, 0.92222222, 0.91666667, 0.96111111,
        0.96111111, 0.96666667, 0.95530726, 0.9273743 , 0.93296089]),
 array([0.9       , 0.97777778, 0.95555556, 0.91666667, 0.95      ,
        0.96666667, 0.97777778, 0.96648045, 0.96089385, 0.93854749])]

In [29]:
np.average(scores)

0.9252940720049658

In [30]:
average_score = []
for i in range(len(scores)):
    average_score.append(np.average(scores[i]))
average_score

[0.8759186840471755,
 0.9343234016139043,
 0.9398975791433891,
 0.9510366232153942]

Here we used cross_val_score to
fine tune our random forest classifier and figured that having around 40 trees in random forest gives best result. 